# -- The producer in this file is using a differnt kafka Topic -- #

In [18]:
# 1. **Import** **libraries**
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [19]:
#Getting the path of the datasets
file_path = "./dataset/SWat_merged.csv"

In [20]:
dataset = pd.read_csv(file_path)

In [21]:
#Change 'A ttack' with Attack in the Attack dataframe

dataset['Normal/Attack'].unique()

dataset['Normal/Attack'].replace({'A ttack': 'Attack'}, inplace=True)

dataset['Normal/Attack'].unique()

array(['Normal', 'Attack'], dtype=object)

In [22]:
#convert timestamp to datatime format
dataset['Timestamp'] = pd.to_datetime(dataset['Timestamp'])



dataset.head()

dataset.shape

/var/folders/t2/x0qtdtj94l15f_fvb24d6k040000gn/T/ipykernel_2428/1035356482.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset['Timestamp'] = pd.to_datetime(dataset['Timestamp'])


(946719, 53)

In [23]:
dataset.head(1)

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,2015-12-22 16:00:00,2.470294,261.5804,2,2,1,244.3284,8.19008,306.101,2.471278,...,1,1,10.02948,0.0,4.277749,0.000256,1,1,1,Normal


In [24]:
df = dataset.set_index('Timestamp')

In [25]:
#Divide the data into two entities
#Feature Matrix
X = df.iloc[:, :-1]
#Dependent varable vector
Y = df.iloc[:, -1]

Y = Y.reset_index(drop=True)


In [26]:
X.head(2)


,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-12-22 16:00:00,2.470294,261.5804,2,2,1,244.3284,8.19008,306.101,2.471278,2,...,0.0,1,1,10.02948,0.0,4.277749,0.000256,1,1,1
2015-12-22 16:00:01,2.457163,261.1879,2,2,1,244.3284,8.19008,306.101,2.468587,2,...,0.0,1,1,10.02948,0.0,4.277749,0.000256,1,1,1


In [27]:

Y.head(2)


0    Normal
1    Normal
Name: Normal/Attack, dtype: object

In [28]:

X.info()
X.shape

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 946719 entries, 2015-12-22 16:00:00 to 2016-02-01 14:59:59
Data columns (total 51 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   FIT101   946719 non-null  float64
 1   LIT101   946719 non-null  float64
 2   MV101    946719 non-null  int64  
 3   P101     946719 non-null  int64  
 4   P102     946719 non-null  int64  
 5   AIT201   946719 non-null  float64
 6   AIT202   946719 non-null  float64
 7   AIT203   946719 non-null  float64
 8   FIT201   946719 non-null  float64
 9   MV201    946719 non-null  int64  
 10  P201     946719 non-null  int64  
 11  P202     946719 non-null  int64  
 12  P203     946719 non-null  int64  
 13  P204     946719 non-null  int64  
 14  P205     946719 non-null  int64  
 15  P206     946719 non-null  int64  
 16  DPIT301  946719 non-null  float64
 17  FIT301   946719 non-null  float64
 18  LIT301   946719 non-null  float64
 19  MV301    946719 non-null  int64

(946719, 51)

In [29]:
# Divide features into numerical and categorical

num_col = []
cat_col = []

for column in X.columns:
    unique_values = X[column].nunique()
    if unique_values > 10:
        num_col.append(column)
    else:
        cat_col.append(column)

print('num column', num_col)
print('num column', cat_col)

print('num column', len(num_col))
print('num column', len(cat_col))

num column ['FIT101', 'LIT101', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'DPIT301', 'FIT301', 'LIT301', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'PIT501', 'PIT502', 'PIT503', 'FIT601']
num column ['MV101', 'P101', 'P102', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302', 'P401', 'P402', 'P403', 'P404', 'UV401', 'P501', 'P502', 'P601', 'P602', 'P603']
num column 25
num column 26


In [49]:

numerical_data = X[num_col]
categorical_data = X[cat_col]

# Min-Max scaling for numerical features
scaler = MinMaxScaler()
numerical_data_scaled = scaler.fit_transform(numerical_data)
numerical_data_scaled_df = pd.DataFrame(numerical_data_scaled, columns=numerical_data.columns)

# One-hot encoding for categorical features
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoder.fit(categorical_data)

categorical_feature_names = encoder.get_feature_names_out(categorical_data.columns)
categorical_data_encoded = encoder.transform(categorical_data)
categorical_data_encoded_df = pd.DataFrame(categorical_data_encoded, columns=categorical_feature_names)
preprocessed_data = pd.concat([numerical_data_scaled_df, categorical_data_encoded_df], axis=1)



In [50]:
preprocessed_data.head(5)

,FIT101,LIT101,AIT201,AIT202,AIT203,FIT201,DPIT301,FIT301,LIT301,AIT401,...,MV303_2,MV304_1,MV304_2,P301_2,P302_2,P402_2,P403_2,UV401_2,P501_2,P602_2
0,0.894987,0.160292,0.730144,0.732892,0.073596,0.874201,0.462186,0.940694,0.182199,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.890230,0.159845,0.730144,0.732892,0.073596,0.873249,0.462186,0.940371,0.182199,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.883848,0.159533,0.730144,0.732892,0.073596,0.872796,0.463182,0.939886,0.182199,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.879786,0.158819,0.730144,0.732892,0.073596,0.872524,0.463182,0.939886,0.182049,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.878510,0.158372,0.731064,0.732892,0.073596,0.872524,0.463182,0.939886,0.181936,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:

preprocessed_data.shape

(946719, 51)

In [51]:
Y = Y.apply(lambda x: 1 if x == 'Attack' else 0)

Y.head()


0    0
1    0
2    0
3    0
4    0
Name: Normal/Attack, dtype: int64

In [52]:
preprocessed_data['Normal/Attack'] = Y[0]

In [56]:
preprocessed_data.head(1)

,FIT101,LIT101,AIT201,AIT202,AIT203,FIT201,DPIT301,FIT301,LIT301,AIT401,...,MV304_1,MV304_2,P301_2,P302_2,P402_2,P403_2,UV401_2,P501_2,P602_2,Normal/Attack
0,0.894987,0.160292,0.730144,0.732892,0.073596,0.874201,0.462186,0.940694,0.182199,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [57]:
preprocessed_data.head()

,FIT101,LIT101,AIT201,AIT202,AIT203,FIT201,DPIT301,FIT301,LIT301,AIT401,...,MV304_1,MV304_2,P301_2,P302_2,P402_2,P403_2,UV401_2,P501_2,P602_2,Normal/Attack
0,0.894987,0.160292,0.730144,0.732892,0.073596,0.874201,0.462186,0.940694,0.182199,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.890230,0.159845,0.730144,0.732892,0.073596,0.873249,0.462186,0.940371,0.182199,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.883848,0.159533,0.730144,0.732892,0.073596,0.872796,0.463182,0.939886,0.182199,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.879786,0.158819,0.730144,0.732892,0.073596,0.872524,0.463182,0.939886,0.182049,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.878510,0.158372,0.731064,0.732892,0.073596,0.872524,0.463182,0.939886,0.181936,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [62]:
preprocessed_data = pd.concat([dataset["Timestamp"], preprocessed_data], axis=1)

In [63]:
preprocessed_data.head()

,Timestamp,FIT101,LIT101,AIT201,AIT202,AIT203,FIT201,DPIT301,FIT301,LIT301,...,MV304_1,MV304_2,P301_2,P302_2,P402_2,P403_2,UV401_2,P501_2,P602_2,Normal/Attack
0,2015-12-22 16:00:00,0.894987,0.160292,0.730144,0.732892,0.073596,0.874201,0.462186,0.940694,0.182199,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2015-12-22 16:00:01,0.890230,0.159845,0.730144,0.732892,0.073596,0.873249,0.462186,0.940371,0.182199,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2015-12-22 16:00:02,0.883848,0.159533,0.730144,0.732892,0.073596,0.872796,0.463182,0.939886,0.182199,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2015-12-22 16:00:03,0.879786,0.158819,0.730144,0.732892,0.073596,0.872524,0.463182,0.939886,0.182049,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2015-12-22 16:00:04,0.878510,0.158372,0.731064,0.732892,0.073596,0.872524,0.463182,0.939886,0.181936,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [67]:
processed_data_path = "./dataset/processed_data.csv"
df.to_csv(processed_data_path, index=False)

Write to kafka Producer

In [66]:
from confluent_kafka import Producer
import json
import time
import pandas as pd

# Kafka configuration
kafka_config = {
    'bootstrap.servers': 'localhost:9092',
    'client.id': 'python-producer'
}

# Kafka topic
topic = 'swat_topic_processed'

# Create Kafka producer
producer = Producer(kafka_config)

# Read from Pandas DataFrame and send to Kafka
i = 0
for _, row in preprocessed_data.iterrows():
    # Convert each row to JSON and send to Kafka
    message = json.dumps(row.to_dict(), default=str).encode('utf-8')
    producer.produce(topic, key='key', value=message)

    # Sleep 
    time.sleep(5)
    #  make sure to comment / uncomment this  stopping condition as per your need
    # i += 1
    # if i == 10:
    #     break

# Close the producer
producer.flush()


%4|1701820276.736|TERMINATE|python-producer#producer-2| [thrd:app]: Producer terminating with 78 messages (91426 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


0